In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
! cp -r /content/drive/My\ Drive/Hackabit/new /content/

^C


In [0]:
# ! for f in /content/new/*.mp4; do mkdir ${f:0:-4}; ffmpeg -i $f -r 1/1 ${f:0:-4}/p%d.png; done;

In [0]:
from keras.applications import Xception
from keras.models import Model
from keras.layers import Dense, Input, Activation
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Nadam
import numpy as np
import matplotlib.pyplot as plt
import os
import keras.preprocessing.image as kimg
from keras import callbacks
import keras.applications as kapp
import keras as ks
import random
import math
from math import sqrt

Using TensorFlow backend.


In [0]:
cnn_base = kapp.ResNet152(input_shape=(240,240,3),weights="imagenet",include_top=False)

x=cnn_base.output
x=ks.layers.Reshape((1,8,8,2048))(x)
x=ks.layers.ConvLSTM2D(256, (2, 2))(x)
x=ks.layers.BatchNormalization()(x)
x=GlobalAveragePooling2D()(x)
x=Dense(1000,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
preds=Dense(2,activation='softmax')(x) #dense layer 2
 #final layer with softmax activation
cnn = Model(input=cnn_base.input, output=preds)
# cnn.trainable = False
# for i in (len(cnn.layers)-1,len(cnn.layers)-5,-1):
for i in range(len(cnn.layers)-5):
    cnn.layers[i].trainable=False
cnn.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 240, 240, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 246, 246, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 120, 120, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 120, 120, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [0]:
# frames = 10
# # encoded_frames = Input((frames, 2048))
# # encoded_sequence = LSTM(1024)(encoded_frames)
# # hidden_layer = Dense(output_dim=512, activation="relu")(encoded_sequence)
# # outputs = Dense(output_dim=2, activation="softmax")(hidden_layer)
# # model = Model([encoded_frames], outputs)
# model = ks.models.Sequential()
# model.add(ks.layers.BatchNormalization(input_shape=(frames,1536)))
# model.add(ks.layers.LSTM(256, return_sequences=True))
# model.add(ks.layers.Reshape((frames*256,)))
# model.add(ks.layers.Dense(1024, activation='relu'))
# model.add(ks.layers.Dense(2, activation='softmax'))
# model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_215 (Bat (None, 10, 1536)          6144      
_________________________________________________________________
lstm_31 (LSTM)               (None, 10, 256)           1836032   
_________________________________________________________________
reshape_7 (Reshape)          (None, 2560)              0         
_________________________________________________________________
dense_66 (Dense)             (None, 1024)              2622464   
_________________________________________________________________
dense_67 (Dense)             (None, 2)                 2050      
Total params: 4,466,690
Trainable params: 4,463,618
Non-trainable params: 3,072
_________________________________________________________________


In [0]:
# optimizer = Nadam(lr=0.002,beta_1=0.9,beta_2=0.999,epsilon=1e-08,schedule_decay=0.004)
cnn.compile(loss="binary_crossentropy", optimizer='adam',metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# def image_generator(path, labels, frames=10, batch_size=32):
#     l = len(os.listdir(path))
#     i = 1
#     batch_x = np.ndarray((batch_size, frames, 240, 240, 3))
#     batch_y = np.ndarray((batch_size, 2))
#     while True:
#         for j in range(batch_size):
#             for k in range(frames):
#                 batch_x[j,k] = kimg.img_to_array(kimg.load_img(os.path.join(path, 'p'+str(i+k)+'.png'), target_size=(240, 240)))/255
#                 batch_y[j] = labels[i+k-1]
#             i += 1

#         if i > l-frames-batch_size:
#             i = 1
#         yield batch_x, batch_y

In [0]:
def load_labels(video):
    with open('/content/drive/My Drive/Hackabit/annotations/' + video + '.txt') as f:
        t = f.readlines()
    t = list(map(lambda x: list(map(lambda y: round(int(y)/25), x.strip().split())), t))
    l = len(os.listdir('/content/drive/My Drive/Hackabit/new/' + video))
    labels = np.ndarray((l, 2))
    for i in range(l):
        chngd = False
        for j in t:
            if i+1>=j[0] and i+1<=j[1]:
                labels[i] = [0,1]
                chngd = True
                break
            if not chngd:
                labels[i] = [1,0]
    return labels, l

In [0]:
from keras.utils import Sequence

class DataSequence(Sequence):
    def __init__(self, video, labels, mode='train'):
        self.video = video
        self.videos = os.listdir('/content/drive/My Drive/Hackabit/new/' + video)
        self.labels = labels
        self.mode = mode
        self.bsz = 32

    def __len__(self):
        # compute number of batches to yield
        return int(math.ceil(len(self.videos) / self.bsz))

    def on_epoch_end(self):
        pass
        # Shuffles indexes after each epoch if in training mode
        # self.indexes = range(len(self.videos))
        # if self.mode == 'train':
            # self.indexes = random.sample(self.indexes, k=len(self.indexes))

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return self.labels[idx * self.bsz: (idx + 1) * self.bsz]

    def get_batch_features(self, idx):
        # Fetch a batch of inputs
        imgs = np.array([kimg.img_to_array(kimg.load_img('/content/drive/My Drive/Hackabit/new/' + self.video + '/'+str(i), target_size=(240, 240)))\
                         for i in self.videos[idx * self.bsz: (1 + idx) * self.bsz]])
        return kapp.inception_v3.preprocess_input(imgs)

    def __getitem__(self, idx):
        batch_x = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_x, batch_y

In [0]:
def cnn_train(video, epochs):
    labels,l = load_labels(video)
    # frames_path = '/content/drive/My Drive/Hackabit/new/' + video
    # frames = os.listdir(frames_path)
  
#   for i in range(l):
    # img = kapp.inception_v3.preprocess_input(np.expand_dims(kimg.img_to_array(kimg.load_img('/content/drive/My Drive/Hackabit/new/' + video + '/' + frames[i],
                                                                                            # target_size=(240, 240))), axis=0))
    cnn.fit(DataSequence(video, labels), epochs=epochs)

In [0]:
# train_data = kimg.ImageDataGenerator(rescale=1/255).flow_from_directory('/content/new/', classes=[video], target_size = (240,240), shuffle=False)
def cnn_predict(l, video):
    predicted = np.ndarray((l, 1000))
    for i in range(1, l+1):
        img = kapp.inception_v3.preprocess_input(np.expand_dims(kimg.img_to_array(kimg.load_img('/content/drive/My Drive/Hackabit/new/' + video + '/p'+str(i)+'.png',
                                                                                                target_size=(240, 240))), axis=0))
        predicted[i-1] = cnn.predict(img)
    return predicted

In [0]:
def build_data(labels, l, predicted):
    num_data = l - frames + 1
    data_x, data_y = np.ndarray((num_data, frames, 1000)), np.ndarray((num_data, 2))
    for i in range(num_data):
        for j in range(frames):
            data_x[i][j] = predicted[i+j]
        data_y[i] = labels[i]
    print(data_x.shape, data_y.shape)
    return data_x, data_y

In [0]:
def video_predictor(video):
    # labels,l = load_labels(video)
    frames_path = '/content/drive/My Drive/Hackabit/new/' + video
    frames = os.listdir(frames_path)
    c0 = 0
    for i in range(len(frames)):
        img = kapp.inception_v3.preprocess_input(np.expand_dims(kimg.img_to_array(kimg.load_img('/content/drive/My Drive/Hackabit/new/' + video + '/' + frames[i],
                                                                                                    target_size=(240, 240))), axis=0))
        pred = cnn.predict(img)
        if pred[0][0] > 0.75:
            c0 += 1
    print(c0/len(frames))
    #   random.shuffle(frames)
    # count = 0
    # opt_threshold = 0.15
    # for i in range(int(sqrt(len(frames)))):
    #     img = kapp.inception_v3.preprocess_input(np.expand_dims(kimg.img_to_array(kimg.load_img('/content/drive/My Drive/Hackabit/new/' + video + '/' + frames[i],
    #                                                                                                 target_size=(240, 240))), axis=0))
    #     pred = cnn.predict(img)
    #     print(pred)
    #     if(pred[0][1]<0.15):
    #         count+=1
    #     else:
    #         opt_threshold = pred[0][1]
    #         break

    # if(count == int(sqrt(len(frames)))):
    #     return "Non-Violent"
    # else:
    #     current = int(sqrt(len(frames))) + 1
    #     while(True):
    #         img = kapp.inception_v3.preprocess_input(np.expand_dims(kimg.img_to_array(kimg.load_img('/content/drive/My Drive/Hackabit/new/' + video + '/' + frames[current],
    #                                                                                                     target_size=(240, 240))), axis=0))
    #         pred = cnn.predict(img)
    #         print(pred)
    #         if(pred[0][1]<opt_threshold):
    #             current = current + 1
    #             if(current>len(frames)):
    #                 return "Non-Violent"
    #         else:
    #             return "Violent"

In [0]:
vids = list(map(lambda x: x[:-4], os.listdir('/content/drive/My Drive/Hackabit/annotations')))
for vid in vids:
  cnn_train(vid, 5)

Epoch 1/5
8/8 [==============================] - 5s 653ms/step - loss: 0.2915 - acc: 0.9722
Epoch 2/5
8/8 [==============================] - 5s 629ms/step - loss: 0.1917 - acc: 0.9960
Epoch 3/5
8/8 [==============================] - 5s 630ms/step - loss: 0.1326 - acc: 0.9921
Epoch 4/5
8/8 [==============================] - 5s 628ms/step - loss: 0.0929 - acc: 1.0000
Epoch 5/5
8/8 [==============================] - 5s 632ms/step - loss: 0.0728 - acc: 0.9921
Epoch 1/5
1/1 [==============================] - 1s 757ms/step - loss: 0.8290 - acc: 0.5000
Epoch 2/5
1/1 [==============================] - 1s 542ms/step - loss: 0.7796 - acc: 0.5000
Epoch 3/5
1/1 [==============================] - 1s 552ms/step - loss: 0.7579 - acc: 0.5000
Epoch 4/5
1/1 [==============================] - 1s 555ms/step - loss: 0.7450 - acc: 0.5000
Epoch 5/5
1/1 [==============================] - 1s 559ms/step - loss: 0.7122 - acc: 0.5000
Epoch 1/5
1/1 [==============================] - 1s 839ms/step - loss: 1.0875 - 

KeyboardInterrupt: ignored

In [0]:
video_predictor("vid1")

0.9053030303030303


In [0]:
# checkpointer = callbacks.ModelCheckpoint(filepath='/content/drive/My\Drive/Hackabit/lstm_weights.hdf5')
# vids = list(map(lambda x: x[:-4], os.listdir('/content/drive/My Drive/Hackabit/annotations')))
# data_x, data_y = [], []
# frames = 10

# for vid in vids:
#     if vid[0]=='.':
#         continue
#     try:
#       labels, l = load_img(vid)
#       print(vid, l, end=' ')
#       if l<frames:
#           continue
#       predicted = cnn_predict(l, vid)
#       x, y = build_data(labels, l, predicted)
#       data_x.append(x)
#       data_y.append(y)
#     except FileNotFoundError:
#       print(vid + " Not found")
#       continue

0egEFZq2Y28 225 (216, 10, 1000) (216, 2)
37Bk1K_0sTU 23 (14, 10, 1000) (14, 2)
2buhK32I9k0 32 (23, 10, 1000) (23, 2)
0f-QmF0Avvc 147 (138, 10, 1000) (138, 2)
1KA8kQS2PTE 49 (40, 10, 1000) (40, 2)
1QHa5yz_qV0 358 (349, 10, 1000) (349, 2)
42aIApcCP6Y 316 (307, 10, 1000) (307, 2)
5hQapiCQJqs 148 (139, 10, 1000) (139, 2)
18GLKynpwvM Not found
5m5uP3QggAY 51 (42, 10, 1000) (42, 2)
5pXwkqiqgHY 85 (76, 10, 1000) (76, 2)
5ZKVh39Y550 200 (191, 10, 1000) (191, 2)
AVIUxloxcEA 153 (144, 10, 1000) (144, 2)
6jSJz9Zfaw8 159 (150, 10, 1000) (150, 2)
as6c4eZSDoE 45 (36, 10, 1000) (36, 2)
5rVrxG3QAh0 230 (221, 10, 1000) (221, 2)
AKlO-kETSvs 152 (143, 10, 1000) (143, 2)
aVs7FuCIMxs 173 (164, 10, 1000) (164, 2)
8JvjJMADF0w 95 (86, 10, 1000) (86, 2)
9lj85YnyWMU 76 (67, 10, 1000) (67, 2)
BC9ry2ZoYks 205 (196, 10, 1000) (196, 2)
dWGJwImOwZg 140 (131, 10, 1000) (131, 2)
bZRtBXtc88M 97 (88, 10, 1000) (88, 2)
CKPBMlM_4k4 126 (117, 10, 1000) (117, 2)
E8bP__QcTtY 103 (94, 10, 1000) (94, 2)
dSB3rx5aNio 62 (53, 10,

In [0]:
# for dx, dy in zip(data_x, data_y):
#     dx = np.expand_dims(dx, axis=0)
#     hist = cnn.fit(dx, dy, epochs=1, callbacks=[checkpointer])
#     # plt.plot(hist.history['loss'])

ValueError: ignored

In [0]:
dx = data_x[100:200]
dy = data_y[100:200]

In [0]:
! cp -r /content/new/ /content/drive/My\ Drive/Hackabit

In [0]:
# np.save('/content/drive/My Drive/Hackabit/data_y', data_y)

import pickle

with open('/content/drive/My Drive/Hackabit/data_yl', 'wb') as fp:
    pickle.dump(data_y, fp)

In [0]:
ds = DataSequence('5ZKVh39Y550', load_labels('5ZKVh39Y550')[0])

In [0]:
print(len(ds))
# ds[2][0].shape

7


In [0]:
cnn_train('0egEFZq2Y28', 5)



Epoch 1/5
8/8 [==============================] - 21s 3s/step - loss: 0.7804 - acc: 0.4841
Epoch 2/5
8/8 [==============================] - 5s 668ms/step - loss: 0.5931 - acc: 0.7977
Epoch 3/5
8/8 [==============================] - 5s 667ms/step - loss: 0.2140 - acc: 0.9484
Epoch 4/5
8/8 [==============================] - 5s 666ms/step - loss: 0.1370 - acc: 0.9921
Epoch 5/5
8/8 [==============================] - 5s 669ms/step - loss: 0.0943 - acc: 0.9921


In [0]:
cnn.save_weights('/content/drive/My Drive/Hackabit/rcl_weights.hdf5')